# Otimizando Modelo de Classificação

In [ ]:
!pip install optimum transformers datasets evaluate huggingface_hub transformers[sklearn] optimum[onnxruntime]

## Modelo


In [ ]:
modelo_id_tc = 'ahmedrachid/FinancialBERT-Sentiment-Analysis'

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(modelo_id_tc)

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification

In [ ]:
modelo_tc = ORTModelForSequenceClassification.from_pretrained(modelo_id_tc, export = True)

## Salvar modelo pré treinado

In [ ]:
dir_tc = 'tmp/onnx-text/'

tokenizer.save_pretrained(dir_tc)
modelo_tc.save_pretrained(dir_tc)

## Inferindo Modelo

In [ ]:
from optimum.pipelines import pipeline

onnx_tc = pipeline('text-classification',model=modelo_tc,tokenizer=tokenizer,accelerator='ort')

In [ ]:
frase = 'Market trends indicate a positive outlook for pharmaceutical companies.'

In [ ]:
def text_class(onnx_pipe,frase=frase):
  saida = onnx_pipe(frase)
  return saida

In [ ]:
text_class(onnx_tc)

# Otimizando o Modelo

In [ ]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

otimizador = ORTOptimizer.from_pretrained(modelo_tc)
config_otim = OptimizationConfig(optimization_level=99, enable_transformers_specific_optimizations=True)

In [ ]:
dir_otm = 'tmp/onnx-otimizado/'

In [ ]:
otimizador.optimize(save_dir=dir_otm,optimization_config=config_otim)

## Inferindo Modelo Otimizado

In [ ]:
modelo_otm = ORTModelForSequenceClassification.from_pretrained(dir_otm,
                                                               file_name = 'model_optimized.onnx')
tokenizer = AutoTokenizer.from_pretrained(dir_otm)

In [ ]:
onnx_otm = pipeline('text-classification', model = modelo_otm, tokenizer = tokenizer, accelerator = 'ort')
text_class(onnx_otm)

## Avaliando Otimização

In [ ]:
dados_id_tc = 'nickmuchi/financial-classification'

In [ ]:
from datasets import load_dataset

data = load_dataset(dados_id_tc, split = 'test')
data

In [ ]:
from evaluate import evaluator

In [ ]:
def avaliacao(data, model_tc):
  eval = evaluator('text-classification')
  resultados = eval.compute(
      model_or_pipeline=model_tc,
      data=data,
      input_column = 'text',
      label_column = 'labels',
      label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
  )
  return resultados

In [ ]:
mod_original = avaliacao(data, onnx_tc)

In [ ]:
mod_otimizado = avaliacao(data, onnx_otm)

In [ ]:
mod_original

In [ ]:
mod_otimizado

# Publicar Modelo

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
nome_rep_otm = 'ONNX-otimizado-financialBERT-Sentiment-Analysis'

In [ ]:
onnx_otm.push_to_hub(nome_rep_otm)

## Usar Modelo Otimizado

In [ ]:
modelo_otimizado = pipeline('text-classification','mirla/ONNX-otimizado-financialBERT-Sentiment-Analysis',
                            accelerator='ort')
text_class(modelo_otimizado)